In [64]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/pollution.csv')
df.head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [65]:
df.isnull().sum()

,0
No,0
year,0
month,0
day,0
hour,0
pm2.5,2067
DEWP,0
TEMP,0
PRES,0
cbwd,0


In [66]:
df.shape

(43824, 13)

In [67]:
df.drop(['year','month',	'day',	'hour'],axis = 1)

,No,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,NaN,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...,...
43819,43820,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,43821,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,43822,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,43823,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [68]:
df.isnull().sum()

,0
No,0
year,0
month,0
day,0
hour,0
pm2.5,2067
DEWP,0
TEMP,0
PRES,0
cbwd,0


In [69]:
df = df.dropna(axis=0)

In [70]:
df.shape

(41757, 13)

In [71]:
X = df.drop(['pm2.5'],axis = 1)
y = df['pm2.5']

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cat_cols = ['cbwd']

le = LabelEncoder()
for col in cat_cols:
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [78]:
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

print("y_train_scaled shape:", y_train_scaled.shape)
print("y_test_scaled shape:", y_test_scaled.shape)

y_train_scaled shape: (33405, 1)
y_test_scaled shape: (8352, 1)


In [85]:
def create_sequences(Xdata, ydata, timesteps):
    X_seq, y_seq = [], []
    for i in range(len(Xdata) - timesteps):
        X_seq.append(Xdata[i:(i + timesteps)])
        y_seq.append(ydata[i + timesteps])
    return np.array(X_seq), np.array(y_seq)

timesteps = 37

X_train_reshaped, y_train_reshaped = create_sequences(X_train_scaled, y_train_scaled, timesteps)
X_test_reshaped, y_test_reshaped = create_sequences(X_test_scaled, y_test_scaled, timesteps)

In [86]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
n_features = X_train_reshaped.shape[2]
model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(timesteps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))

model.add(Dense(1))
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 37, 50)         │        12,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 37, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 37, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 37, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,051 (207.23 KB)

 Trainable params: 53,051 (207.23 KB)

 Non-trainable params: 0 (0.00 B)

In [88]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])


In [ ]:
history = model.fit(X_train_reshaped, y_train_reshaped, epochs=20, batch_size=32, verbose=1)


Epoch 1/20
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 60s 52ms/step - loss: 0.0093 - mean_absolute_error: 0.0716
Epoch 2/20
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 55s 53ms/step - loss: 0.0087 - mean_absolute_error: 0.0701
Epoch 3/20
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 54s 52ms/step - loss: 0.0085 - mean_absolute_error: 0.0695
Epoch 4/20
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 55s 53ms/step - loss: 0.0086 - mean_absolute_error: 0.0696
Epoch 5/20
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 55s 53ms/step - loss: 0.0087 - mean_absolute_error: 0.0699
Epoch 6/20
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 56s 54ms/step - loss: 0.0086 - mean_absolute_error: 0.0696
Epoch 7/20
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 55s 52ms/step - loss: 0.0086 - mean_absolute_error: 0.0693
Epoch 8/20
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 55s 53ms/step - loss: 0.0085 - mean_absolute_error: 0.0696
Epoch 9/20
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 81s 52ms/step - loss: 0.0086 - mean_absolute_error: 0.0692
Epoch 10/20
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 56s 53ms/step - loss: 0.0086 - mean_absolute_err